## 6. flask 다양한 기능

### 6.1. 에러(error) 다루기

* errorhandler를 사용하여 HTTP 오류 코드가 나오는 페이지를 정의할 수 있음
  - return 의 두번째 인자로 에러코드를 넘겨주지 않으면 200 성공으로 인지함
  
> HTTP 응답코드와 프로토콜에 대해서는 별도 챕터에서 설명

In [ ]:
@app.errorhandler(404)  # 없는 페이지를 요청했을 때의 에러
def page_not_found(error):
    return "<h1>404 Error</h1>", 404

In [ ]:
from flask import Flask 
import requests

app = Flask(__name__)   

@app.errorhandler(404)
def page_not_found(error):
    return "<h1>404 Error</h1>", 404

@app.route("/google")
def get_google():
    response = requests.get("http://www.google.co.kr")
    return response.text 

if __name__ == "__main__":
    app.run(host="0.0.0.0", port="8080")

 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Apr/2018 21:59:09] "GET / HTTP/1.1" 404 -


### 6.2. 로깅(logging) 다루기
- 서버는 24시간 동작하므로, 문제가 있을 때, 어떤 문제가 있었는지, 파악하기 위해 로깅 기능을 사용함
- 사용자 모니터링, 해킹 확인등 다양한 상용화시 문제에 대해서도 로깅 기능을 활용할 수 있음

> 로깅은 개발 단계에서는 크게 필요하지 않으므로, 가볍게 참고로만 알아둠 <br>
> 추후 상용화 단계에서 추가 기능을 넣어서 구현 예정

### 간단한 logging 라이브러리 사용법

- 파이썬에는 로그를 다루는 logging 라이브러리가 있음
- 로딩 정보는 레벨이 있음
- 로깅 정보는 로그의 레벨에 따라서 출력을 제한 할 수 있음
  - DEBUG > INFO > WARNING > ERROR > Critical

In [3]:
import logging
# 파일로 남기기 위해서는 filename='test.log' 파라미터, 어느 로그까지 남길 것인지를 level 로 설정 가능
logging.basicConfig(filename='test.log', level=logging.ERROR)

# 로그를 남길 부분에 다음과 같이 로그 레벨에 맞추어 출력해주면 해당 내용이 파일에 들어감
logging.debug("debug")
logging.info("info")
logging.warning("warning")
logging.error("error")
logging.critical("critical")

### flask 와 logging
- logging 라이브러리와 함께 flask logging 기능 사용 가능
- 주요 logging 핸들러 (어떻게 로그를 다룰 것인지에 대해 미리 구현된 함수들을 제공)
  - FileHandler - 파일로 로그를 저장
  - RotatingFileHandler - 파일로 로그를 저장하되, 파일이 정해진 사이즈를 넘어가면, 새로운 파일로 생성
    - maxBytes=하나의파일사이즈, backupCount=파일갯수
    - 전체 파일을 다 쓰면, 다시 처음부터 씀
  - NTEventLogHandler - 윈도우 시스템 로그로 남김
  - SysLogHandler - 유닉스 계열 시스템의 syslog 로 남김

> 서버 상에서는 로그 파일이 전체 디스크를 채울 경우, 비정상동작을 할 수 있으므로 RotatingFileHandler 가 일반적인 경우에는 적합 

- 다음과 같이 debug 옵션을 써줄 경우, app.debug 에 해당 값이 들어가고, debug=True 일 경우, 상세 정보를 화면에 표시
  - 주로 개발/테스트시 사용

```python
    app.run(host="0.0.0.0", port="8080", debug=True)
```

In [2]:
from flask import Flask 
import requests

app = Flask(__name__)


app.debug = False

if not app.debug:
    import logging
    from logging.handlers import RotatingFileHandler # logging 핸들러 이름을 적어줌
    file_handler = RotatingFileHandler('dave_server.log', maxBytes=2000, backupCount=10) 
    file_handler.setLevel(logging.WARNING) # 어느 단계까지 로깅을 할지를 적어줌
    app.logger.addHandler(file_handler) # app.logger.addHandler() 에 등록시켜줘야 app.logger 로 사용 가능

@app.errorhandler(404)
def page_not_found(error):
    app.logger.error(error)
    return "<h1>404 Error</h1>", 404

if __name__ == "__main__":
    app.run(host="0.0.0.0", port="8080",debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.112:8080/ (Press CTRL+C to quit)
[2024-04-12 23:44:47,437] ERROR in 828915146: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.
192.168.1.112 - - [12/Apr/2024 23:44:47] "GET / HTTP/1.1" 404 -
[2024-04-12 23:44:52,235] ERROR in 828915146: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.
192.168.1.112 - - [12/Apr/2024 23:44:52] "GET /test HTTP/1.1" 404 -


<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">실습하기</font><br>
http://localhost:8080/test 로 테스트해보기
</div>

### 6.3. 다양한 데코레이터

* before_first_request : 웹 application 기동 이후 가장 처음에 들어오는 HTTP 요청에서만 실행
* before_request : HTTP 요청이 들어올 때마다 실행
  - before_first_request, before_request는 인자를 전달할 수는 없음 <br><br>  
* after_request : HTTP 요청 처리가 끝나고 브라우저에 응답하기 전에 실행
  - response 를 리턴해야 함

In [ ]:
from flask import Flask 
import requests

app = Flask(__name__)   

@app.before_first_request
def before_first_request():
    print ("flask 실행 후 첫 요청 때만 실행")
 
@app.before_request
def before_request():
    print ("HTTP 요청이 들어올 때마다 실행")
 
@app.after_request
def after_request(response):
    print ("HTTP 요청 처리가 끝나고 브라우저에 응답하기 전에 실행")
    return response

    
@app.route("/hello")
def hello():                           
    return "<h1>Hello Flask!</h1>"


if __name__ == "__main__":
    app.run(host="0.0.0.0", port="8080")